# PORT protocol showcase

This notebook shows `PORT` protocol from the network perspective, for comparison with `TYPHOON`.

Some of the system packages required for this notebook, including: `ip-route`, `tc`.

Use the following cell to configure PORT hyperparameters with environmental variables.

> NB! It should be run *prior* to all the other cells, since hyperparameters are being resolved only once upon `PortCore` class import.

In [ ]:
from os import environ

environ["PROTOCOL_NAME"] = "port"
environ["SEASIDE_LOG_LEVEL"] = "DEBUG"
environ["PROTOCOL_MIN_TIMEOUT"] = environ["PORT_MIN_TIMEOUT"] = str(16)
environ["PROTOCOL_MAX_TIMEOUT"] = environ["PORT_MAX_TIMEOUT"] = str(32)
environ["PROTOCOL_DEFAULT_TIMEOUT"] = environ["PORT_DEFAULT_TIMEOUT"] = str(24)
environ["PROTOCOL_MIN_NEXT_IN"] = environ["PORT_MIN_NEXT_IN"] = str(8.0)
environ["PROTOCOL_MAX_NEXT_IN"] = environ["PORT_MAX_NEXT_IN"] = str(32.0)
environ["PROTOCOL_MAX_RETRIES"] = environ["PORT_MAX_RETRIES"] = str(5)

This notebook requires some extra capabilities being applied to your `python` executable and also added to the `JuPyter` server process.
Don't worry: everything can be done just in a few steps.

> Why are these capabilities needed?
> Well, let's say for "safety" reasons.
> We do not want to apply **tc** (traffic control) commands directly to the loopback interface, right?
> That's why we need a pair of virtual interfaces with virtual IPs (`veth`s), so that we can apply fine-grained filters.
> And creating these interfaces requires some permissions.

First of all, the following cell should be executed before any other notebook cells (it might throw an error!):

In [ ]:
from pathlib import Path
from sys import path

path.append(str(Path() / ".."))

from typhoon.utils import configure_ambient_permissions

configure_ambient_permissions()

If the previous cell has thrown an error, that means that your `python` executable does not have `CAP_NET_ADMIN` and `CAP_NET_RAW` right now.
This can be solved by either of these three solutions:

1. Run the server in a `Docker` container (either `privileged` or with `CAP_NET_ADMIN` set) - the dockerfile is *not* provided.
2. Run the following cell to patch your current python executable (don't forget to run the next cell when you're done!).
3. Run the server with `sudo` (might be dangerous and not recommended!).

Whatever way you have chosen, the kernel should be restarted.

In [ ]:
from typhoon.utils import toggle_network_permissions

toggle_network_permissions(True)

Run this cell after you finished working with the notebook - it will unpatch you `python` executable.

In [ ]:
from typhoon.utils import toggle_network_permissions

toggle_network_permissions(False)

> NB! In every cell below, only `PORT` packets are considered.
> Other `TCP` service packets are also captured, but not decoded.
> As a result, all non-`PORT` packets **will be marked as `INVALID`**.

## What is `PORT`?

`PORT` is a `TCP`-based protocol, that ensures connection stability using `TCP_KEEPALIVE`.
Whereas some of the `PORT` properties (data packet lengths and contents) are randomized, most of them (`TCP` packet contents and sending time) behave predictably.

#### Graph description

Packet length will be displayed in bar charts below as *bar heights*.

Packet delivery offset is shown on horizontal axis as *offset from the connection termination*.
I.e. value of -300000ms means that packet was sent 5 minutes before the connection was closed.

Packets that *are not part of `PORT` protocol* (e.g. `TCP` `SYN`, `ACK`, etc.) will be shown and described as **invalid**.

### `PORT` guarantees

Since `PORT` is TCP-based, **all** the lost or corrupted packets will be retransmitted, it ensures safe data delivery.

## Ideal connection

Simple sanity check here.
Client connects to echo server, sends it a small package, receives response, checks it's the same and exits.

Client and server exchange initialization packets, than data is transmitted.
Apart from that, multiple `TCP`-originated packets are being sent here and there.

As a result, a small sequence diagram is produced, showing packet exchange, their direction, size and timestamp.

In [ ]:
from sources.protocol.port_socket import PortListener, PortClient
from sources.protocol.utils import ProtocolReturnCode

from typhoon.utils import GraphConf, sniff, LISTENER_KEY, CLIENT_KEY, USER_TOKEN


async def echo_server_callback(user_id: int, data: bytes) -> bytes:
    print(f"Received data from client {user_id}: {data}")
    return data


async def success_listener_callback(name: str, token: bytes) -> ProtocolReturnCode:
    print(f"Initialized client '{name}' with token: {token}")
    return ProtocolReturnCode.SUCCESS


async with sniff(GraphConf()) as s:
    async with PortListener(LISTENER_KEY, s.server_address).ctx(success_listener_callback, echo_server_callback) as l:
        async with PortClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx() as c:
            request = b"Hi server!"
            await c.write(request)
            response = await c.read()
            assert(request == response)

Stale connection is shown here.
Client is connected to server, waits for some time and disappears.
Server tries to restore connection, but fails and closes it after some time.

Client and server exchange initialization packets, then several successful `TCP` keepalive `ACK`s are being sent (small invalid packets, below 100 bytes in size).
Finally, some unsuccessful `ACK`s attempts by server should be present before the connection termination.

As a result, a small packet plot is produced, showing packet exchange, their type, size and timestamp.

In [ ]:
from asyncio import sleep
from os import environ

from sources.protocol.port_socket import PortListener, PortClient

from typhoon.utils import PlotConf, sniff, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

SLEEP_DELAY = (float(environ["PORT_MAX_NEXT_IN"]) + float(environ["PORT_MAX_TIMEOUT"])) * 5
TIMEOUT_DELAY = float(environ["PORT_MAX_NEXT_IN"]) * (float(environ["PORT_MAX_RETRIES"]) + 1)


async with sniff(PlotConf()) as s:
    async with PortListener(LISTENER_KEY, s.server_address).ctx(data_callback=lambda _: None) as l:
        async with PortClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(lambda _: None, False) as c:
            print(f"Connection established! Sleeping for {SLEEP_DELAY}...")
            await sleep(SLEEP_DELAY)
        print(f"Connection interrupted! Waiting for {TIMEOUT_DELAY}...")
        await sleep(TIMEOUT_DELAY)

Successful connection is shown here.
Client is connected to server, they exchange data packets every second.
In the end, the connection is terminated successfully.

In the beginning two packets should be initialization packets, in the end two packets should be termination packets.
In between, multiple data and `TCP` keepalive `ACK` packages should be observable.

As a result, a small packet plot is produced, showing packet exchange, their type and size.
The receiving time is omitted.

In [ ]:
from sources.protocol.port_socket import PortListener, PortClient

from typhoon.utils import PlotConf, sniff, SequenceConfig, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

CONFIG = SequenceConfig(messages=96, quiet=True, client_sequence=lambda _: 1, server_sequence=lambda _: 0)


async with sniff(PlotConf(expand_time=False)) as s:
    async with PortListener(LISTENER_KEY, s.server_address).ctx(data_callback=CONFIG.seq_server_callback) as l:
        async with PortClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(CONFIG.seq_client_callback) as c:
            await CONFIG.process(c)

## Network in chaos

This section shows how `PORT` protocol behaves under pressure of unfriendly network conditions.

Here, network conditions are ideal.

In [ ]:
from sources.protocol.port_socket import PortListener, PortClient

from typhoon.utils import PlotConf, sniff, SequenceConfig, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

CONFIG = SequenceConfig()


async with sniff(PlotConf()) as s:
    async with PortListener(LISTENER_KEY, s.server_address).ctx(data_callback=CONFIG.seq_server_callback) as l:
        async with PortClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(CONFIG.seq_client_callback) as c:
            await CONFIG.process(c)

Here, random jitter between 25 and 175 ms is applied to packets.

In [ ]:
from asyncio import sleep

from sources.protocol.port_socket import PortListener, PortClient

from typhoon.utils import PlotConf, sniff, SequenceConfig, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

CONFIG = SequenceConfig()


async with sniff(PlotConf(), r"delay 100ms 75ms") as s:
    async with PortListener(LISTENER_KEY, s.server_address).ctx(data_callback=CONFIG.seq_server_callback) as l:
        async with PortClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(CONFIG.seq_client_callback) as c:
            await CONFIG.process(c)
            await sleep(CONFIG.base_delay() * 4)

Here, 30% of packets will be dropped.

In [ ]:
from sources.protocol.port_socket import PortListener, PortClient

from typhoon.utils import PlotConf, sniff, SequenceConfig, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

CONFIG = SequenceConfig()


async with sniff(PlotConf(), r"loss 30%") as s:
    async with PortListener(LISTENER_KEY, s.server_address).ctx(data_callback=CONFIG.seq_server_callback) as l:
        async with PortClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(CONFIG.seq_client_callback) as c:
            await CONFIG.process(c)

Here, 30% of packets will be duplicated.

In [ ]:
from sources.protocol.port_socket import PortListener, PortClient

from typhoon.utils import PlotConf, sniff, SequenceConfig, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

CONFIG = SequenceConfig()


async with sniff(PlotConf(), r"duplicate 30%") as s:
    async with PortListener(LISTENER_KEY, s.server_address).ctx(data_callback=CONFIG.seq_server_callback) as l:
        async with PortClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(CONFIG.seq_client_callback) as c:
            await CONFIG.process(c)

Here, some of the packets will be reordered within a 125ms timeframe.

In [ ]:
from asyncio import sleep

from sources.protocol.port_socket import PortListener, PortClient

from typhoon.utils import PlotConf, sniff, SequenceConfig, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

CONFIG = SequenceConfig()


async with sniff(PlotConf(), r"delay 125ms reorder 30% 50%") as s:
    async with PortListener(LISTENER_KEY, s.server_address).ctx(data_callback=CONFIG.seq_server_callback) as l:
        async with PortClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(CONFIG.seq_client_callback) as c:
            await CONFIG.process(c)
            await sleep(CONFIG.base_delay() * 2.5)

Here, 30% of packets will be corrupted.

In [ ]:
from sources.protocol.port_socket import PortListener, PortClient

from typhoon.utils import PlotConf, sniff, SequenceConfig, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

CONFIG = SequenceConfig(quiet=True)


async with sniff(PlotConf(), r"corrupt 30%") as s:
    async with PortListener(LISTENER_KEY, s.server_address).ctx(data_callback=CONFIG.seq_server_callback) as l:
        async with PortClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(CONFIG.seq_client_callback) as c:
            await CONFIG.process(c)